In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
import os 
import torchvision.transforms.functional as tvF
from model import UNet
from Config import Config as conf
import time
from dataset_processing import Training_Dataset
from torch.utils.data import Dataset, DataLoader

In [3]:
if __name__ == '__main__':
    """Train the model"""
    if torch.cuda.is_available():
        print('CUDA is available. Training on GPU')
        device = torch.device('cuda')
    else:
        print('CUDA is not available. Training on CPU')
        device = torch.device('cpu')

    model = UNet(conf.img_channel, conf.img_channel)
    model = model.to(device)
    optimizer = Adam(model.parameters(), lr=conf.learning_rate)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    criterion = nn.MSELoss()

    dataset = Training_Dataset(conf.data_path_train, conf.gaussian_noise_param, conf.crop_img_size)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    model.train()

#     conf.max_epoch
    for epoch in range(1):
        for i, (x_i, y_i) in enumerate(dataloader):
            x_i = x_i.to(device)
            y_i = y_i.to(device)
            optimizer.zero_grad()
            output = model(x_i)
            loss = criterion(output, y_i)
            loss.backward()
            optimizer.step()
            if epoch % conf.save_per_epoch == 0:
                print(f'Epoch {epoch}, batch {i}, loss {loss.item()}')
        scheduler.step()
        print(f'Epoch {epoch} finished')
        # if epoch % conf.save_per_epoch == 0:
        #     save_model(model, epoch)

CUDA is available. Training on GPU
Epoch 0, batch 0, loss 0.19387850165367126
Epoch 0, batch 1, loss 0.19817084074020386
Epoch 0, batch 2, loss 0.1506856381893158
Epoch 0, batch 3, loss 0.13109348714351654
Epoch 0, batch 4, loss 0.07362194359302521
Epoch 0, batch 5, loss 0.048002295196056366
Epoch 0, batch 6, loss 0.03851987421512604
Epoch 0, batch 7, loss 0.03445399925112724
Epoch 0, batch 8, loss 0.026197802275419235
Epoch 0, batch 9, loss 0.026701873168349266
Epoch 0, batch 10, loss 0.02241823449730873
Epoch 0, batch 11, loss 0.0239744633436203
Epoch 0, batch 12, loss 0.020129259675741196
Epoch 0, batch 13, loss 0.02304907701909542
Epoch 0, batch 14, loss 0.018388286232948303
Epoch 0, batch 15, loss 0.022014793008565903
Epoch 0, batch 16, loss 0.02022387459874153
Epoch 0, batch 17, loss 0.018599525094032288
Epoch 0, batch 18, loss 0.01791894994676113
Epoch 0, batch 19, loss 0.01811176910996437
Epoch 0, batch 20, loss 0.015513445250689983
Epoch 0, batch 21, loss 0.016972392797470093


In [10]:
from dataset_processing import Testing_Dataset

In [11]:
if torch.cuda.is_available():
    print('CUDA is available. Training on GPU')
    device = torch.device('cuda')
else:
    print('CUDA is not available. Training on CPU')
    device = torch.device('cpu')

#load the model
# model = UNet(in_channels=conf.img_channel,out_channels=conf.img_channel)
# model.load_state_dict(torch.load(conf.model_path_test))
# model.eval()
# model.to(device)

test_dataset = Testing_Dataset(conf.data_path_test,conf.crop_img_size)
test_loader =  DataLoader(test_dataset, batch_size=1, shuffle=False)

save_dir = conf.denoised_dir
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for i, x_i in enumerate(test_loader):
    x_i = x_i.to(device)
    output = model(x_i).detach().cpu()

    output = tvF.to_pil_image(torch.clamp(output.squeeze(0), 0, 1))
    fname = os.path.splitext(test_loader.dataset.test_list[i])[0]
    output.save(os.path.join(save_dir, f'{fname}-denoised.png'))

CUDA is available. Training on GPU


ValueError: too many values to unpack (expected 2)